## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [2]:
import numpy as np
import os
from random import shuffle
import re
from collections import Counter

In [3]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [4]:
import urllib.request
import zipfile
import lxml.etree

In [5]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [6]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [7]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [8]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [9]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [10]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [11]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [12]:
len(sentences_ted)

266694

In [18]:
print(sentences_ted[0])
print(sentences_ted[1])


['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


AttributeError: 'list' object has no attribute 'shape'

### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [37]:
# ...

c = Counter([y for x in sentences_ted for y in x])
counts_ted_top1000 = [x[1] for x in c.most_common(1000)]
print(counts_ted_top1000)

[207748, 149305, 125169, 114818, 105399, 95146, 83180, 78070, 74738, 70923, 67629, 63251, 57156, 49241, 37014, 33102, 30806, 29713, 27995, 27344, 26732, 26519, 25962, 24706, 23377, 22757, 21246, 21041, 20201, 19488, 19216, 19021, 18626, 17928, 17908, 17551, 17012, 16723, 16273, 15868, 15452, 14387, 14061, 13986, 13917, 13896, 13882, 13864, 13278, 12879, 12363, 12302, 12163, 11935, 11595, 11576, 11506, 11445, 11366, 10902, 10900, 10463, 10446, 10180, 10179, 10091, 10029, 10021, 9911, 9872, 9675, 9376, 9312, 9239, 9160, 8906, 8841, 8823, 8629, 8262, 8256, 8011, 7858, 7735, 7724, 7617, 7537, 7536, 7281, 7227, 7183, 7168, 7141, 7119, 6937, 6806, 6521, 6424, 6358, 6307, 6292, 6229, 6207, 5972, 5907, 5695, 5684, 5571, 5552, 5501, 5456, 5379, 5374, 5312, 5275, 5220, 5197, 5175, 5153, 5133, 4935, 4929, 4919, 4660, 4546, 4544, 4511, 4485, 4454, 4399, 4299, 4143, 4132, 4130, 4120, 4079, 4034, 3971, 3968, 3799, 3798, 3791, 3787, 3786, 3707, 3611, 3494, 3449, 3364, 3346, 3301, 3245, 3217, 3201, 31

Plot distribution of top-1000 words

In [38]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

/home/osboxes/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  """Entry point for launching an IPython kernel.


### Part 3: Train Word2Vec

In [39]:
from gensim.models import Word2Vec

In [40]:
# ...
model_ted = Word2Vec(sentences_ted, min_count=10)

In [51]:
len(model_ted.wv.vocab)

14427

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [42]:
model_ted.most_similar("man")

/home/osboxes/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/osboxes/anaconda3/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.8463952541351318),
 ('guy', 0.824913740158081),
 ('girl', 0.7642336487770081),
 ('boy', 0.7619692087173462),
 ('lady', 0.7520687580108643),
 ('david', 0.7102624773979187),
 ('soldier', 0.7011938095092773),
 ('gentleman', 0.7008219361305237),
 ('poet', 0.6912198066711426),
 ('kid', 0.687893807888031)]

In [43]:
model_ted.most_similar("computer")

/home/osboxes/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/osboxes/anaconda3/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('machine', 0.7458442449569702),
 ('software', 0.7268987894058228),
 ('robot', 0.7182953357696533),
 ('device', 0.6782342791557312),
 ('3d', 0.6656662821769714),
 ('program', 0.6439570188522339),
 ('chip', 0.6298129558563232),
 ('camera', 0.6239331364631653),
 ('interface', 0.6209791898727417),
 ('test', 0.6184127330780029)]

In [ ]:
# ...

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [53]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_ted = [x[0] for x in c.most_common(1000)]
words_top_vec_ted = model_ted[words_top_ted]

/home/osboxes/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [54]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [55]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [56]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [57]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [58]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [59]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


In [71]:
sentences_token_wiki = []
for sent_str in sentences_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_token_wiki.append(tokens)

Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [73]:
# ...
c = Counter([y for x in sentences_token_wiki for y in x])
counts_wiki_top1000 = [x[1] for x in c.most_common(1000)]
print(c.most_common(1000))

[('the', 1286909), ('of', 547311), ('and', 499176), ('in', 433935), ('to', 399238), ('a', 358293), ('was', 215283), ('s', 164469), ('on', 153563), ('as', 145060), ('for', 140511), ('that', 138041), ('with', 129424), ('by', 126099), ('is', 103287), ('his', 92834), ('at', 92806), ('he', 91714), ('it', 89124), ('from', 88826), ('were', 70688), ('an', 64198), ('had', 58530), ('which', 55034), ('be', 47827), ('this', 43297), ('but', 42644), ('are', 41033), ('first', 40906), ('not', 39562), ('their', 38795), ('after', 38448), ('one', 38041), ('her', 37494), ('also', 37053), ('its', 36655), ('they', 35020), ('two', 34300), ('or', 33451), ('have', 32962), ('who', 31835), ('new', 30591), ('been', 30374), ('has', 29845), ('she', 29612), ('when', 27027), ('during', 26490), ('time', 26285), ('other', 25469), ('would', 25190), ('into', 24905), ('all', 24355), ('i', 23673), ('more', 22930), ('while', 21546), ('over', 21417), ('him', 19900), ('game', 19859), ('only', 19661), ('later', 19564), ('most'

In [74]:
model_wiki = Word2Vec(sentences_token_wiki, min_count=10)

#### t-SNE visualization

In [75]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_wiki = [x[0] for x in c.most_common(1000)]
words_top_vec_wiki = model_wiki.wv[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [76]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)